In [6]:
import cv2
import matplotlib.pyplot as plt
from keras import backend as K
from keras.models import load_model

In [8]:
model = load_model('BrainTumor10Epochs.keras')

AttributeError: 'Adam' object has no attribute 'build'

In [ ]:

def generate_saliency_map(model, input_image):
    """
    Generates a saliency map for a given input image and model.
    
    :param model: Trained Keras model
    :param input_image: Input image for which the saliency map is to be generated
    :return: Saliency map
    """
    # Convert the image to array and expand dimensions to match the model's input shape
    input_image = np.expand_dims(input_image, axis=0)
    
    # Define the output and input tensor for the model
    output_tensor = model.output
    input_tensor = model.input
    
    # Compute the gradient of the output with respect to the input
    grads = K.gradients(output_tensor, input_tensor)[0]
    
    # Define a Keras function to calculate the saliency map
    saliency_function = K.function([input_tensor, K.learning_phase()], [grads])
    
    # Compute the saliency map
    saliency_map = saliency_function([input_image, 0])[0]
    
    # Remove the batch dimension and normalize the saliency map
    saliency_map = np.squeeze(saliency_map)
    saliency_map = np.abs(saliency_map)
    saliency_map = saliency_map / np.max(saliency_map)
    
    return saliency_map

# Generate saliency map for a sample input image from the test set
sample_image = x_test[0]  # Replace with the appropriate image index
saliency_map = generate_saliency_map(model, sample_image)

# Display the original image and the saliency map
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(sample_image)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Saliency Map')
plt.imshow(saliency_map, cmap='hot')
plt.axis('off')

plt.tight_layout()
plt.show()
